# Laden der Daten
In diesem Notebook werden die Standorte der schweizer Handyantennen aus der Datenquelle data.geo.admin.ch eingelesen. Überflüssige spalten werden entfernt, koordinaten werden ins WSG84 format umgewandelt. Und die Adressdaten werden per Nominatim abgerufen, verarbeitet und im Dataframe erweitert

## Verwendete Links/Quellen

Letzter Zugriff: 01.12.2021

In [11]:
url_5g="http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-5g/data/ch.bakom.mobil-antennenstandorte-5g_de.json"
url_lte='http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-lte/data/ch.bakom.mobil-antennenstandorte-lte_de.json'
url_umts='http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-umts/data/ch.bakom.mobil-antennenstandorte-umts_de.json'
url_gsm='http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-gsm/data/ch.bakom.mobil-antennenstandorte-gsm_de.json'
url_plz = 'https://swisspost.opendatasoft.com/explore/dataset/plz_verzeichnis_v2/table/?disjunctive.postleitzahl&geofilter.distance='


***

# Installation notwendiger Bibliotheken

In [2]:
!pip install --upgrade pip
!pip install folium
!pip install matplotlib
!pip install geopandas
!pip install geopy
!pip install pyproj
!pip install certifi
!pip install ssl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached ssl-1.16.tar.gz (33 kB)

  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'



  exit code: 1
  
  [25 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 14, in <module>
    File "C:\ProgramData\Anaconda3\lib\site-packages\setuptools\__init__.py", line 189, in <module>
      monkey.patch_all()
    File "C:\ProgramData\Anaconda3\lib\site-packages\setuptools\monkey.py", line 99, in patch_all
      patch_for_msvc_specialized_compiler()
    File "C:\ProgramData\Anaconda3\lib\site-packages\setuptools\monkey.py", line 169, in patch_for_msvc_specialized_compiler
      patch_func(*msvc14('_get_vc_env'))
    File "C:\ProgramData\Anaconda3\lib\site-packages\setuptools\monkey.py", line 149, in patch_params
      mod = import_module(mod_name)
    File "C:\ProgramData\Anaconda3\lib\importlib\__init__.py", line 127, in import_module
      return _bootstrap._gcd_import(name[level:], package, level)
    File "C:\ProgramData\Anaconda3\lib\site-packages\setuptools\_distutils\_msvccompiler.py", li

***

# Module importieren

In [2]:
import csv

# library to handle data in a vectorized manner 
import numpy as np 

# library for data analsysis
import pandas as pd

import geopandas as gpd

# library to handle JSON files
import json

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

import folium
from folium import plugins

# library to handle requests
import requests

import zipfile

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# necessary for loading url data
import io

from geopy.geocoders import Nominatim

import ssl
import certifi

import geopy.geocoders

import time


from pyproj import Proj, transform

print('Libraries imported.')



from geopy.exc import GeocoderTimedOut

from shapely.geometry import Point, LineString

Libraries imported.


***

# Funktionen

In [3]:
# https://www.swisstopo.admin.ch/content/swisstopo-internet/de/online/calculation-services/_jcr_content/contentPar/tabs/items/dokumente_und_publik/tabPar/downloadlist/downloadItems/8_1467103085694.download/refsys_d.pdf)
# Zugriff: 08.04.2023
def convert_lv95_wgs84(e,n): #Funktion Umrechnung lv95 zu wgs84
    
    y = (e - 2600000)/1000000
    x = (n - 1200000)/1000000
    
    l = 2.6779094 + 4.728982 * y + 0.791484 * y * x + 0.1306 * y * x**2 - 0.0436 * y**3
    b = 16.9023892 + 3.238272 * x - 0.270978 * y**2 - 0.002528 * x**2 - 0.0447 * y**2 * x - 0.0140 * x**3
    
    laenge_wgs84 = l * 100 / 36
    breite_wgs84 = b * 100 / 36
    
    return breite_wgs84, laenge_wgs84

Die Klasse load_data wurde implementiert, um alle Daten mit einem Befehl einlesen zu können. <br>


In [4]:
class load_data:
    def __init__(self,path):
        self.path=path

    @staticmethod
    def load_from_url(url):
        s=requests.get(url).content
        df=pd.read_csv(io.StringIO(s.decode('utf-8')))
        return df

        
    @staticmethod
    def load_pkl(path_to_pkl):
        with open(path_to_pkl,'rb') as f:
            data=pickle.load(f)
        return data
        
    @staticmethod
    def load_csv(path_to_csv):
        df=pd.read_csv(path_to_csv,sep=',')
        return df

***

# 1) Import mapping dictionary for postal codes
<br>Dieser Datensatz liefert dieschweizer Postleitzahlen mit den dazugehöhrigen Ortsnamen dazaus wird ein dictionary erstellt, um später alle Ortsnamen nachzuschlagen <br/>

In [5]:
plz=pd.read_csv('./Daten/plz_verzeichnis_v2.csv',sep=';')
plz = plz[['POSTLEITZAHL', 'ORTBEZ18']]
plz = plz.rename({'POSTLEITZAHL': 'PLZ', 'ORTBEZ18': 'ORT'}, axis=1)  # new method
plz = plz.set_index('PLZ').to_dict()['ORT']


# Import locations of antena
<br> JSON from antenna to Pandas <br>

In [32]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-5g_de.json'
antennen_5g = gpd.read_file(data_path)
antennen_5g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2688674.000 1211488.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Sehr Klein,POINT (2666065.000 1214355.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2722651.000 1271244.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2570323.000 1164067.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Klein,POINT (2613427.000 1262637.000)


In [34]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-lte_de.json'
antennen_4g = gpd.read_file(data_path)
antennen_4g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Gross,POINT (2558525.000 1167925.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Gross,POINT (2648322.000 1246675.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Gross,POINT (2503428.000 1117642.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2601088.000 1200806.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Gross,POINT (2567350.000 1228990.000)


In [36]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-umts_de.json'
antennen_3g = gpd.read_file(data_path)
antennen_3g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2601088.000 1200806.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Gross,POINT (2639748.000 1232218.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2613427.000 1262637.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2503428.000 1117642.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Klein,POINT (2570323.000 1164067.000)


In [37]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-gsm_de.json'
antennen_2g = gpd.read_file(data_path)
antennen_2g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2701702.000 1265811.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Mittel,POINT (2618398.000 1227312.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2609644.000 1213344.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2560331.000 1182192.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Sehr Klein,POINT (2557146.000 1210825.000)


# Prepare Datasets
<br> Set column names and drop usless Columns <br>

In [39]:

columns=['language', 'description', 'id', 'power', 'cordinates']
antennen_5g.columns=columns
antennen_4g.columns=columns
antennen_3g.columns=columns
antennen_2g.columns=columns

In [40]:
antennen_5g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_4g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_3g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_2g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_4g.head(4)

,id,power,cordinates
0,1,Gross,POINT (2558525.000 1167925.000)
1,2,Gross,POINT (2648322.000 1246675.000)
2,3,Gross,POINT (2503428.000 1117642.000)
3,4,Mittel,POINT (2601088.000 1200806.000)


# convert coordinates of a dataframe from LV95 to WGS84


In [42]:
# LV95 zu WGS84
def convert_panda_lv95_wgs84(dataFrame ): #Funktion Umrechnung lv95 zu wgs84
    inProj = Proj('epsg:2056')
    outProj = Proj('epsg:4326')
    t = dataFrame['cordinates'].x
    x1,y1 = dataFrame['cordinates'].x, dataFrame['cordinates'].y
    x2,y2 = transform(inProj, outProj, x1,y1)
    dataFrame['AccidentLocation_WGS84_E'] = x2
    dataFrame['AccidentLocation_WGS84_N'] = y2
    return dataFrame

In [43]:
# LV95 zu WGS84
inProj = Proj('epsg:2056')
outProj = Proj('epsg:4326')
x1,y1 = antennen_5g['cordinates'].x, antennen_5g['cordinates'].y
x2,y2 = transform(inProj, outProj, x1,y1)


C:\Users\nikki\AppData\Local\Temp\ipykernel_21872\258762374.py:5: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj, outProj, x1,y1)


# get address data from openstreet map

In [14]:
import ssl
!pip install certifi
import certifi
print(certifi.where())
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())

Defaulting to user installation because normal site-packages is not writeable
C:\ProgramData\Anaconda3\lib\site-packages\certifi\cacert.pem
C:\ProgramData\Anaconda3\lib\site-packages\certifi\cacert.pem


In [15]:
antennen_5g['AccidentLocation_WGS84_E'] = x2
antennen_5g['AccidentLocation_WGS84_N'] = y2
antennen_5g.head()

,id,power,cordinates,AccidentLocation_WGS84_E,AccidentLocation_WGS84_N
0,1,Mittel,POINT (2688674.000 1211488.000),47.048492,8.605623
1,2,Sehr Klein,POINT (2666065.000 1214355.000),47.076917,8.308525
2,3,Mittel,POINT (2722651.000 1271244.000),47.580429,9.069008
3,4,Mittel,POINT (2570323.000 1164067.000),46.627189,7.051134
4,5,Klein,POINT (2613427.000 1262637.000),47.514341,7.616871


In [48]:
antennen_5g = convert_panda_lv95_wgs84(antennen_5g)
antennen_4g = convert_panda_lv95_wgs84(antennen_4g)
antennen_3g = convert_panda_lv95_wgs84(antennen_3g)
antennen_2g = convert_panda_lv95_wgs84(antennen_2g)




C:\Users\nikki\AppData\Local\Temp\ipykernel_21872\948803134.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj, outProj, x1,y1)
C:\Users\nikki\AppData\Local\Temp\ipykernel_21872\948803134.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj, outProj, x1,y1)
C:\Users\nikki\AppData\Local\Temp\ipykernel_21872\948803134.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj, outProj, x1,y1)
C:\Users\nikki\AppData\Local\Temp\ipykernel_21872\948803134.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj, outProj, x1,y1)


In [17]:
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())
geolocator = Nominatim(user_agent="test")
location = geolocator.reverse("52.509669, 13.376294")
location

C:\ProgramData\Anaconda3\lib\site-packages\certifi\cacert.pem


Location(Steinecke, Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, Deutschland, (52.5098134, 13.37631790998454, 0.0))

In [18]:
from geopy.exc import GeocoderTimedOut
import ssl

context = ssl.create_default_context()
der_certs = context.get_ca_certs(binary_form=True)
pem_certs = [ssl.DER_cert_to_PEM_cert(der) for der in der_certs]

with open('wincacerts.pem', 'w') as outfile:
    for pem in pem_certs:
        outfile.write(pem + '\n')
print outfile.where())
def get_kreis(coordinates, attempt=1, max_attempts=5):
    try:
        locator = Nominatim(user_agent="1_Dataset_Load_cleansing")

        location = locator.reverse(coordinates)
     #   return location.address.split(",")[-6].strip() if len(location.address.split(",")) > 6 else location.address.split(",")[-5].strip()
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_kreis(coordinates, attempt=attempt+1)
        raise

SyntaxError: invalid syntax (3412181519.py, line 11)

In [19]:
def get_adress(coordinates, attempt=1, max_attempts=7):
    try:
        locator = Nominatim(user_agent="add_location_adress")
        location = locator.reverse(coordinates)
        address = location.address.split(",")
        countries = address[len(address)-1].split("/")
        address[len(address)-1] = countries[0]
        return address
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_kreis(coordinates, attempt=attempt+1)
        raise

In [ ]:
for index, row in antennen_5g.iterrows():
    print(row['AccidentLocation_WGS84_E'])

In [20]:
antennen_5g['building'] = None
antennen_5g['streetNumber'] = None
antennen_5g['street'] = None
antennen_5g['town'] = None
antennen_5g['district'] = None
antennen_5g['state'] = None


street_number = []
street = []
town = []
postal = []
district = []
state = []
country = []

counter = 0
for index, row in antennen_5g.iterrows():
  #  print(row['AccidentLocation_WGS84_E'])
    coordinates = [row['AccidentLocation_WGS84_E'],row["AccidentLocation_WGS84_N"]]
 #   print(row.dtype)
 #   print(row)

    forward = 0

    adresse = get_adress(coordinates)

    print(adresse)
    if adresse[forward][0].isdigit() or len(adresse[forward]) >1 and adresse[0][1].isdigit():
        antennen_5g.loc[index , "streetNumber"] = adresse[0]
        street_number.append(adresse[0])
        forward += 1

        # ['a', ' Chemin du Genévrier', ' Saint-Légier-La Chiésaz', ' Blonay - Saint-Légier', ' District de la Riviera-Pays-d’Enhaut', ' Vaud', ' 1806', ' Schweiz']
    elif adresse[forward+1][0].isdigit() or len(adresse[forward+1]) >1 and adresse[forward+1][1].isdigit():
        antennen_5g.loc[index , "streetNumber"] = adresse[forward+1]
        antennen_5g.loc[index , "building"] = adresse[forward]
        print(adresse[forward])
        forward += 2

    else:
        street_number.append(None)


    street.append(adresse[forward])
    antennen_5g.loc[index , "street"] = adresse[forward]


    backward = len(adresse)-1

    country.append(adresse[backward])
    antennen_5g.loc[index , "country"] = adresse[backward]

    backward -= 1



    if len(adresse[backward]) == 4 and adresse[backward][0].isdigit():
        antennen_5g.loc[index , "postal"] = adresse[backward]
        antennen_5g.loc[index , "town"] = plz.get(int(adresse[backward]))
    elif len(adresse[backward+1]) == 4 and adresse[backward+1][0].isdigit():
        antennen_5g.loc[index , "postal"] = adresse[backward+1]
        antennen_5g.loc[index , "town"] = plz.get(int(adresse[backward+1]))
        backward = backward + 1
    elif len(adresse[backward-1]) == 4 and adresse[backward-1][0].isdigit():
        antennen_5g.loc[index , "postal"] = adresse[backward-1]
        antennen_5g.loc[index , "town"] = plz.get(int(adresse[backward-1]))
        backward = backward - 1


    backward += 1


    postal.append(adresse[backward])







    backward -= 1


    state.append(adresse[backward])
    antennen_5g.loc[index , "state"] = adresse[backward]
    backward -= 1


    if adresse[backward].__contains__("ezirk") or adresse[backward].__contains__("istrict") or adresse[backward].__contains__("istretto") or adresse[backward].__contains__("erwaltungsregion") or adresse[backward].__contains__("égion administrative") or adresse[backward].__contains__("ahlkreis") or adresse[backward].__contains__("Region"):
        district.append(adresse[backward])
        antennen_5g.loc[index , "district"] = adresse[backward]
        backward -= 1
    else:
        district.append(None)

    town.append(adresse[backward])

    backward -= 1


  #  print(row)

   ## antennen_5g['street'] = adresse[0]

  #  print(int(adresse[0]))




    counter += 1
    if counter % 200 == 0:
        time.sleep(30)




antennen_5g.head()


['Frauholzstrasse', ' Oberes Halti', ' Steinen', ' Schwyz', ' 6422', ' Schweiz']
['Frauholzstrasse', ' Oberes Halti', ' Steinen', ' Schwyz', ' 6422', ' Schweiz']
['A14', ' Milchhof', ' Ebikon', ' Luzern', ' 6030', ' Schweiz']
['A14', ' Milchhof', ' Ebikon', ' Luzern', ' 6030', ' Schweiz']
['Muggenwinggel', ' Weinfelderstrasse', ' Feldhof', ' Märstetten', ' Bezirk Weinfelden', ' Thurgau', ' 8560', ' Schweiz']
['Muggenwinggel', ' Weinfelderstrasse', ' Feldhof', ' Märstetten', ' Bezirk Weinfelden', ' Thurgau', ' 8560', ' Schweiz']
['71', ' Rue de la Léchère', ' la Léchère', ' Bulle', ' District de la Gruyère', ' Fribourg/Freiburg', ' 1630', ' Schweiz']
['71', ' Rue de la Léchère', ' la Léchère', ' Bulle', ' District de la Gruyère', ' Fribourg/Freiburg', ' 1630', ' Schweiz']
['27', ' Weidenstrasse', ' Münchenstein', ' Bezirk Arlesheim', ' Basel-Landschaft', ' 4142', ' Schweiz']
['27', ' Weidenstrasse', ' Münchenstein', ' Bezirk Arlesheim', ' Basel-Landschaft', ' 4142', ' Schweiz']
['Outlet

,id,power,cordinates,AccidentLocation_WGS84_E,AccidentLocation_WGS84_N,building,streetNumber,street,town,district,state,country
0,1,Mittel,POINT (2688674.000 1211488.000),47.048492,8.605623,None,None,Frauholzstrasse,None,None,6422,Schweiz
1,2,Sehr Klein,POINT (2666065.000 1214355.000),47.076917,8.308525,None,A14,Milchhof,None,None,6030,Schweiz
2,3,Mittel,POINT (2722651.000 1271244.000),47.580429,9.069008,None,None,Muggenwinggel,None,None,8560,Schweiz
3,4,Mittel,POINT (2570323.000 1164067.000),46.627189,7.051134,None,71,Rue de la Léchère,None,None,1630,Schweiz
4,5,Klein,POINT (2613427.000 1262637.000),47.514341,7.616871,None,27,Weidenstrasse,None,None,4142,Schweiz


In [ ]:

antennen_5g.head(30)

In [24]:
def add_address(data_frame ):

    data_frame['building'] = None
    data_frame['streetNumber'] = None
    data_frame['street'] = None
    data_frame['town'] = None
    data_frame['district'] = None
    data_frame['state'] = None


    street_number = []
    street = []
    town = []
    postal = []
    district = []
    state = []
    country = []

    counter = 0
    for index, row in data_frame.iterrows():
        coordinates = [row['AccidentLocation_WGS84_E'],row["AccidentLocation_WGS84_N"]]
        forward = 0

        adresse = get_adress(coordinates)

        print(adresse)
        if adresse[forward][0].isdigit() or len(adresse[forward]) >1 and adresse[0][1].isdigit():
            data_frame.loc[index , "streetNumber"] = adresse[0]
            street_number.append(adresse[0])
            forward += 1
        elif adresse[forward+1][0].isdigit() or len(adresse[forward+1]) >1 and adresse[0][1].isdigit():
            data_frame.loc[index , "streetNumber"] = adresse[forward+1]
            data_frame.loc[index , "building"] = adresse[forward]
            forward += 2

        else:
            street_number.append(None)


        street.append(adresse[forward])
        data_frame.loc[index , "street"] = adresse[forward]


        backward = len(adresse)-1

        country.append(adresse[backward])
        data_frame.loc[index , "country"] = adresse[backward]

        backward -= 1



        if len(adresse[backward]) == 4 and adresse[backward][0].isdigit():
            data_frame.loc[index , "postal"] = adresse[backward]
            data_frame.loc[index , "town"] = plz.get(int(adresse[backward]))
        elif len(adresse[backward+1]) == 4 and adresse[backward+1][0].isdigit():
            data_frame.loc[index , "postal"] = adresse[backward+1]
            data_frame.loc[index , "town"] = plz.get(int(adresse[backward+1]))
            backward = backward + 1
        elif len(adresse[backward-1]) == 4 and adresse[backward-1][0].isdigit():
            data_frame.loc[index , "postal"] = adresse[backward-1]
            data_frame.loc[index , "town"] = plz.get(int(adresse[backward-1]))
            backward = backward - 1


        backward += 1


        postal.append(adresse[backward])







        backward -= 1


        state.append(adresse[backward])
        data_frame.loc[index , "state"] = adresse[backward]
        backward -= 1


        if adresse[backward].__contains__("ezirk") or adresse[backward].__contains__("istrict"):
            district.append(adresse[backward])
            data_frame.loc[index , "district"] = adresse[backward]
            backward -= 1
        else:
            district.append(None)

        town.append(adresse[backward])

        backward -= 1


    #  print(row)

    ## data_frame['street'] = adresse[0]

    #  print(int(adresse[0]))




        counter += 1
        if counter % 150 == 0:
            time.sleep(30)


    return data_frame






In [ ]:
antennen_5g = add_address(antennen_5g)
antennen_4g = add_address(antennen_4g)
antennen_3g = add_address(antennen_3g)
antennen_2g = add_address(antennen_2g)

['55', ' Route de la Chapelle', ' La Pierraz', ' Chavannes les-Forts', ' Siviriez', ' District de la Glâne', ' Fribourg/Freiburg', ' 1676', ' Schweiz']
['64', ' Bernstrasse West', ' Helgenfeld', ' Suhr', ' Bezirk Aarau', ' Aargau', ' 5034', ' Schweiz']
['2', ' Avenue des Amazones', ' La Gradelle', ' Chêne-Bougeries', ' Genève', ' 1224', ' Schweiz']
['12b', ' Breitenrainstrasse', ' Breitenrain', ' Stadtteil V', ' Bern', ' Verwaltungskreis Bern-Mittelland', ' Verwaltungsregion Bern-Mittelland', ' Bern/Berne', ' 3013', ' Schweiz']
['21', " Grand'Rue", ' Esserts', ' Les Breuleux', ' District des Franches-Montagnes', ' Jura', ' 2345', ' Schweiz']
['276', ' Schaffhauserstrasse', ' Oerlikon', ' Kreis 11', ' Zürich', ' Bezirk Zürich', ' Zürich', ' 8057', ' Schweiz']
['Chavornay', ' District du Jura-Nord vaudois', ' Vaud', ' 1373', ' Schweiz']
['12', ' Bruggmatte', ' Reiden', ' Luzern', ' 6260', ' Schweiz']
['3', ' Chemin des Coquelicots', ' Châtelaine', ' Vernier', ' Genève', ' 6643', ' Schwei

In [22]:
# antennen_5g.to_csv('./Daten/5gAntennen.csv',sep=';', index=False)
# antennen_4g.to_csv('./Daten/4gAntennen.csv',sep=';', index=False)
antennen_3g.to_csv('./Daten/3gAntennen.csv',sep=';', index=False)
antennen_2g.to_csv('./Daten/2gAntennen.csv',sep=';', index=False)

In [ ]:
## 1.5) Reverse Geocoding - Quartier & Stadtkreis

Im Folgenden werden die ersten Reinigunsschritte getätigt. Die Angaben der Unfaelle sind auf 4 verschiedenen Sprachen <br> 
verfügbar. Es werden allerdings nur die Hauptspalten benötigt, weswegen die anderen Sprachen fallen gelassen werden können. <br> 
Die 'AccidentType_de' Spalte wird allrdings einer neuen Spalte zugewiesen, da die dort enthalten Informationen <br> 
wichtig sind und nur umbenannt.<br><br>

Uns interessieren ausserdem nur die Unfaelle die in 2019 vorkamen, sowie Unfaelle in die ein Fahrrad<br>
involviert war. Diese werden gefiltert und die Umlaute in der 'AccidentExpl' ersetzt.

In [34]:
# Überflüssige Spalten löschen
try:
    df_unfaelle=df_unfaelle.rename({'AccidentType_de':'AccidentExpl'},axis=1)
    df_unfaelle['AccidentWeekDay']=df_unfaelle['AccidentWeekDay_de']
    df_unfaelle['RoadType']=df_unfaelle['RoadType_de']
    df_unfaelle['AccidentSeverityCategory']=df_unfaelle.AccidentSeverityCategory_de
    df_unfaelle=df_unfaelle.drop(columns=list(df_unfaelle.filter(regex='(_fr|_it|_en|_de|Code|text|UID)')))
    df_unfaelle=df_unfaelle.drop(columns='AccidentType')

except:
    print('Already Dropped')

In [35]:
# Filtern nach Jahr == 2019 und Velofahrer in Unfall verwickelt == True
df_unfaelle = df_unfaelle[(df_unfaelle.AccidentInvolvingBicycle == True) & (df_unfaelle.AccidentYear == 2019)]

In [36]:
# Umschreiben der Umlaute in ae usw.
df_unfaelle=df_unfaelle.replace({'AccidentExpl':{'ä':'ae','Ä':'Ae','Ü':'Ue','ü':'ue','Ö':'Oe','ö':'oe'}},regex=True)

In [37]:
# Index Reset
df_unfaelle.reset_index(inplace=True,drop=True)
df_unfaelle.head()

,AccidentExpl,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,AccidentYear,AccidentMonth,AccidentWeekDay,AccidentHour
0,Frontalkollision,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682347,1247786,2019,1,Freitag,13
1,Einbiegeunfall,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2680457,1247471,2019,1,Montag,6
2,Ueberqueren der Fahrbahn,Unfall mit Sachschaden,False,True,False,Hauptstrasse,2681804,1246614,2019,1,Montag,18
3,Abbiegeunfall,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682226,1247798,2019,1,Dienstag,7
4,Ueberqueren der Fahrbahn,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2682856,1247821,2019,1,Dienstag,19


In [38]:
df_unfaelle.isnull().values.any() # Übeprüfung missing values

False

Umwandeln der Koordinaten mithilfe der Funktion.

In [39]:
df_unfaelle['Latidute_wgs84'],df_unfaelle['Longitude_wgs84'] = zip(*df_unfaelle.apply(lambda x: convert_lv95_wgs84(x.AccidentLocation_CHLV95_E,x.AccidentLocation_CHLV95_N), axis=1))
df_unfaelle

,AccidentExpl,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,AccidentYear,AccidentMonth,AccidentWeekDay,AccidentHour,Latidute_wgs84,Longitude_wgs84
0,Frontalkollision,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682347,1247786,2019,1,Freitag,13,47.375761,8.529004
1,Einbiegeunfall,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2680457,1247471,2019,1,Montag,6,47.373162,8.503925
2,Ueberqueren der Fahrbahn,Unfall mit Sachschaden,False,True,False,Hauptstrasse,2681804,1246614,2019,1,Montag,18,47.365288,8.521601
3,Abbiegeunfall,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682226,1247798,2019,1,Dienstag,7,47.375884,8.527405
4,Ueberqueren der Fahrbahn,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2682856,1247821,2019,1,Dienstag,19,47.376012,8.535750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,Ueberholunfall oder Fahrstreifenwechsel,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2682340,1244588,2019,12,Freitag,12,47.347001,8.528324
691,Abbiegeunfall,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682804,1247769,2019,12,Sonntag,4,47.375551,8.535052
692,Ueberqueren der Fahrbahn,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2680387,1247556,2019,12,Sonntag,19,47.373935,8.503014
693,Schleuder- oder Selbstunfall,Unfall mit Sachschaden,False,True,True,Nebenstrasse,2683523,1247624,2019,12,Dienstag,12,47.374156,8.544544


Einlesen als Geopandas DataFrame und umwandeln der Koordinaten mithilfe <br>
von Geopandas in eine Geometrie.

In [40]:
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df_unfaelle.AccidentLocation_CHLV95_E, df_unfaelle.AccidentLocation_CHLV95_N)]
gdf_unfaelle = gpd.GeoDataFrame(df_unfaelle, geometry=geometry,crs='EPSG:2056')
gdf_unfaelle=gdf_unfaelle.to_crs(epsg=4326)
gdf_unfaelle.head(5)

,AccidentExpl,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,AccidentYear,AccidentMonth,AccidentWeekDay,AccidentHour,Latidute_wgs84,Longitude_wgs84,geometry
0,Frontalkollision,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682347,1247786,2019,1,Freitag,13,47.375761,8.529004,POINT (8.52901 47.37576)
1,Einbiegeunfall,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2680457,1247471,2019,1,Montag,6,47.373162,8.503925,POINT (8.50393 47.37317)
2,Ueberqueren der Fahrbahn,Unfall mit Sachschaden,False,True,False,Hauptstrasse,2681804,1246614,2019,1,Montag,18,47.365288,8.521601,POINT (8.52160 47.36529)
3,Abbiegeunfall,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682226,1247798,2019,1,Dienstag,7,47.375884,8.527405,POINT (8.52741 47.37589)
4,Ueberqueren der Fahrbahn,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2682856,1247821,2019,1,Dienstag,19,47.376012,8.535750,POINT (8.53575 47.37602)


In [41]:
# Überflüssige Spalten löschen
gdf_unfaelle.drop(gdf_unfaelle.filter(regex=r'(CHLV95)').columns, axis=1, inplace=True)
gdf_unfaelle.head(5)

,AccidentExpl,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentYear,AccidentMonth,AccidentWeekDay,AccidentHour,Latidute_wgs84,Longitude_wgs84,geometry
0,Frontalkollision,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2019,1,Freitag,13,47.375761,8.529004,POINT (8.52901 47.37576)
1,Einbiegeunfall,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2019,1,Montag,6,47.373162,8.503925,POINT (8.50393 47.37317)
2,Ueberqueren der Fahrbahn,Unfall mit Sachschaden,False,True,False,Hauptstrasse,2019,1,Montag,18,47.365288,8.521601,POINT (8.52160 47.36529)
3,Abbiegeunfall,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2019,1,Dienstag,7,47.375884,8.527405,POINT (8.52741 47.37589)
4,Ueberqueren der Fahrbahn,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2019,1,Dienstag,19,47.376012,8.535750,POINT (8.53575 47.37602)


In [42]:
gdf_unfaelle.isnull().values.any() # Übeprüfung missing values

False

In [43]:
gdf_unfaelle.to_csv('./Daten/velounfaelle_2019_ZH.csv',sep=';', index=False)

***

# 2) Daten der automatischen Fussgänger- und Velozählung - Viertelstundenwerte der Stadt Zürich 2019
<br>Die Daten zeigen die gezählten Frequenzen je Viertelstunde und Richtung<br/>

In [44]:
df_verkehrszaehlung = load_data.load_from_url(url_zaehl)
df_verkehrszaehlung.head()

,FK_ZAEHLER,FK_STANDORT,DATUM,VELO_IN,VELO_OUT,FUSS_IN,FUSS_OUT,OST,NORD
0,Y2G14045587,61,2019-01-01T00:00,0.0,NaN,NaN,NaN,2683447,1247063
1,U15G3063872,32,2019-01-01T00:00,NaN,NaN,0.0,15.0,2683511,1248166
2,ECO09113502,9,2019-01-01T00:00,0.0,0.0,NaN,NaN,2684578,1251967
3,ECO06040592,3918,2019-01-01T00:00,4.0,6.0,NaN,NaN,2683573,1248545
4,U15G4025180,3278,2019-01-01T00:00,NaN,NaN,1.0,4.0,2680439,1249930


Umbennen der Spaltennamen

In [45]:
columns=['zaehler', 'standort', 'datum', 'velo_in', 'velo_out', 'fuss_in',
       'fuss_out', 'ost', 'nord']
df_verkehrszaehlung.columns=columns
df_verkehrszaehlung.head()

,zaehler,standort,datum,velo_in,velo_out,fuss_in,fuss_out,ost,nord
0,Y2G14045587,61,2019-01-01T00:00,0.0,NaN,NaN,NaN,2683447,1247063
1,U15G3063872,32,2019-01-01T00:00,NaN,NaN,0.0,15.0,2683511,1248166
2,ECO09113502,9,2019-01-01T00:00,0.0,0.0,NaN,NaN,2684578,1251967
3,ECO06040592,3918,2019-01-01T00:00,4.0,6.0,NaN,NaN,2683573,1248545
4,U15G4025180,3278,2019-01-01T00:00,NaN,NaN,1.0,4.0,2680439,1249930


In [46]:
df_verkehrszaehlung['Latidute_wgs84'],df_verkehrszaehlung['Longitude_wgs84'] = zip(*df_verkehrszaehlung.apply(lambda x: convert_lv95_wgs84(x.ost,x.nord), axis=1))
#df_verkehrszaehlung

Das Dateiformat wird in ein Datetime format umgewandelt und dann die jeweiligen Informationen zu <br>
Jahr, Tag, Stunde,Minute in eigene Spalten geschrieben.

In [47]:
df_verkehrszaehlung['datum']=pd.to_datetime(df_verkehrszaehlung.datum)
df_verkehrszaehlung['year']=pd.to_datetime(df_verkehrszaehlung.datum).dt.year
df_verkehrszaehlung['month']=pd.to_datetime(df_verkehrszaehlung.datum).dt.month
df_verkehrszaehlung['day']=pd.to_datetime(df_verkehrszaehlung.datum).dt.day
df_verkehrszaehlung['hour']=pd.to_datetime(df_verkehrszaehlung.datum).dt.hour
df_verkehrszaehlung['minute']=pd.to_datetime(df_verkehrszaehlung.datum).dt.minute
df_verkehrszaehlung.head(4)

,zaehler,standort,datum,velo_in,velo_out,fuss_in,fuss_out,ost,nord,Latidute_wgs84,Longitude_wgs84,year,month,day,hour,minute
0,Y2G14045587,61,2019-01-01,0.0,NaN,NaN,NaN,2683447,1247063,47.369120,8.543433,2019,1,1,0,0
1,U15G3063872,32,2019-01-01,NaN,NaN,0.0,15.0,2683511,1248166,47.379032,8.544486,2019,1,1,0,0
2,ECO09113502,9,2019-01-01,0.0,0.0,NaN,NaN,2684578,1251967,47.413080,8.559332,2019,1,1,0,0
3,ECO06040592,3918,2019-01-01,4.0,6.0,NaN,NaN,2683573,1248545,47.382433,8.545378,2019,1,1,0,0


In [48]:
try:
    df_verkehrszaehlung.drop(['ost','nord'], axis=1, inplace=True)
except:
    print('already dropped')

In [49]:
df_verkehrszaehlung=df_verkehrszaehlung.fillna(0)
df_verkehrszaehlung.head(5)

,zaehler,standort,datum,velo_in,velo_out,fuss_in,fuss_out,Latidute_wgs84,Longitude_wgs84,year,month,day,hour,minute
0,Y2G14045587,61,2019-01-01,0.0,0.0,0.0,0.0,47.369120,8.543433,2019,1,1,0,0
1,U15G3063872,32,2019-01-01,0.0,0.0,0.0,15.0,47.379032,8.544486,2019,1,1,0,0
2,ECO09113502,9,2019-01-01,0.0,0.0,0.0,0.0,47.413080,8.559332,2019,1,1,0,0
3,ECO06040592,3918,2019-01-01,4.0,6.0,0.0,0.0,47.382433,8.545378,2019,1,1,0,0
4,U15G4025180,3278,2019-01-01,0.0,0.0,1.0,4.0,47.395279,8.504129,2019,1,1,0,0


Und umgewandelt als Geopandas mit geometry:

In [50]:
from shapely.geometry import Point

In [51]:
geometry = [Point(xy) for xy in zip(df_verkehrszaehlung.Longitude_wgs84, df_verkehrszaehlung.Latidute_wgs84)]

In [52]:
gdf_verkehrszaehlung = gpd.GeoDataFrame(df_verkehrszaehlung, geometry=geometry,crs='EPSG:4326')

In [53]:
gdf_verkehrszaehlung=gdf_verkehrszaehlung.to_crs(epsg=4326)

In [54]:
gdf_verkehrszaehlung.head(5)

,zaehler,standort,datum,velo_in,velo_out,fuss_in,fuss_out,Latidute_wgs84,Longitude_wgs84,year,month,day,hour,minute,geometry
0,Y2G14045587,61,2019-01-01,0.0,0.0,0.0,0.0,47.369120,8.543433,2019,1,1,0,0,POINT (8.54343 47.36912)
1,U15G3063872,32,2019-01-01,0.0,0.0,0.0,15.0,47.379032,8.544486,2019,1,1,0,0,POINT (8.54449 47.37903)
2,ECO09113502,9,2019-01-01,0.0,0.0,0.0,0.0,47.413080,8.559332,2019,1,1,0,0,POINT (8.55933 47.41308)
3,ECO06040592,3918,2019-01-01,4.0,6.0,0.0,0.0,47.382433,8.545378,2019,1,1,0,0,POINT (8.54538 47.38243)
4,U15G4025180,3278,2019-01-01,0.0,0.0,1.0,4.0,47.395279,8.504129,2019,1,1,0,0,POINT (8.50413 47.39528)


Abspeichern als csv:

In [55]:
gdf_verkehrszaehlung.to_csv('./Daten/verkehrszaehlung_2019_ZH.csv',sep=';', index=False)

***

# 3) Fuss und Fahrradwegnetz Stadt Zürich
<br>Das Fuss- und Velowegnetz beinhaltet alle Fuss- und Velowege in der Stadt Zürich. Es bildet die Grundlage für den städtischen Routenplaner.<br/>
<br>Zugriff: 01:11:2021/20:05 https://data.stadt-zuerich.ch/dataset/geo_fuss__und_velowegnetz<br/>

In [56]:
zip_ref = zipfile.ZipFile('./Daten/Fuss_Velo_Wegnetz.zip')
zip_ref.extractall('./Daten/')
zip_ref.close()

In [57]:
data_path='./Daten/Fuss_Velo_Wegnetz/data/taz_mm.tbl_routennetz.json'
gdf_routennetz = gpd.read_file(data_path)

In [58]:
gdf_routennetz.head()

,id1,velo,velostreifen,veloweg,einbahn,fuss,name,map_velo,map_fuss,se_anno_cad_data,objectid,geometry
0,137,0,0,0,0,1,"Bahnhof Oerlikon, Perron Gleis 6",0,0,NaN,1,"LINESTRING (8.54406 47.41199, 8.54443 47.41212)"
1,138,0,0,0,0,0,Bahnhofstrasse,0,0,NaN,2,"LINESTRING (8.53985 47.37686, 8.53986 47.37688..."
2,139,1,0,0,0,0,Zwischenbächen,1,0,NaN,3,"LINESTRING (8.48085 47.38451, 8.48058 47.38413)"
3,140,1,0,0,0,0,Schanzackerstrasse,0,0,NaN,4,"LINESTRING (8.54727 47.39015, 8.54757 47.39119..."
4,141,0,0,0,0,1,Rotbuchstrasse Überquerung,0,0,NaN,5,"LINESTRING (8.53742 47.39127, 8.53742 47.39141)"


In [59]:
gdf_routennetz.drop(['se_anno_cad_data','id1','objectid'], axis=1, inplace=True)
gdf_routennetz.head(4)

,velo,velostreifen,veloweg,einbahn,fuss,name,map_velo,map_fuss,geometry
0,0,0,0,0,1,"Bahnhof Oerlikon, Perron Gleis 6",0,0,"LINESTRING (8.54406 47.41199, 8.54443 47.41212)"
1,0,0,0,0,0,Bahnhofstrasse,0,0,"LINESTRING (8.53985 47.37686, 8.53986 47.37688..."
2,1,0,0,0,0,Zwischenbächen,1,0,"LINESTRING (8.48085 47.38451, 8.48058 47.38413)"
3,1,0,0,0,0,Schanzackerstrasse,0,0,"LINESTRING (8.54727 47.39015, 8.54757 47.39119..."


In [60]:
gdf_routennetz=gdf_routennetz.replace({'name':{'ä':'ae','ö':'oe','ü':'ue','Ü':'Ue','Ä':'Ae'}},regex=True)

In [61]:
gdf_routennetz.to_csv('./Daten/routennetz_ZH.csv',sep=';', index=False)

Erster Plot zum anschauen des Netzes:

In [62]:
# mail von Maurizio, 24.11.21, workaround for SSL Error:
import ssl
import certifi
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())

/opt/conda/lib/python3.10/site-packages/certifi/cacert.pem


In [71]:
#Breiten- und Längengrad von Zürich
address = 'Zurich, CH'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate of Zurich, CH are {}, {}.'.format(latitude, longitude))

In [72]:
m=folium.Map(location=[latitude,longitude],zoom_start=12)

In [73]:
folium.Choropleth(gdf_routennetz[gdf_routennetz.velostreifen!='0'],line_color='blue',line_weight=5,line_opacity=0.3,columns=['name'],legend_name='hehe').add_to(m)
folium.Choropleth(gdf_routennetz[gdf_routennetz.veloweg!=0].geometry,line_color='red',line_weight=5,line_opacity=0.3).add_to(m)

In [74]:
display(m)

***

# 4) Abbiegeverbote Stadt Zürich
<br>Die Abbiegeverbote geben an, wo man als Velofahrerin nicht abbiegen darf.<br/>
<br>Zugriff: 01:11:2021/20:05 https://data.stadt-zuerich.ch/dataset/geo_fuss__und_velowegnetz<br/>

In [75]:
gdf_abbiegeverbot = gpd.read_file('./Daten/Fuss_Velo_Wegnetz/data/taz_mm.tbl_routennetz_abbiegeverbote.json')
gdf_abbiegeverbot.head()

,id1,se_anno_cad_data,objectid,geometry
0,108,NaN,1,"LINESTRING (8.54526 47.36752, 8.54522 47.36750..."
1,109,NaN,2,"LINESTRING (8.54259 47.37874, 8.54255 47.37867)"
2,110,NaN,3,"LINESTRING (8.55336 47.40247, 8.55345 47.40251..."
3,111,NaN,4,"LINESTRING (8.50459 47.40565, 8.50460 47.40563..."
4,112,NaN,5,"LINESTRING (8.53681 47.38516, 8.53679 47.38521..."


In [76]:
gdf_abbiegeverbot.drop(['se_anno_cad_data','id1','objectid'], axis=1, inplace=True)

In [77]:
gdf_abbiegeverbot.head()

,geometry
0,"LINESTRING (8.54526 47.36752, 8.54522 47.36750..."
1,"LINESTRING (8.54259 47.37874, 8.54255 47.37867)"
2,"LINESTRING (8.55336 47.40247, 8.55345 47.40251..."
3,"LINESTRING (8.50459 47.40565, 8.50460 47.40563..."
4,"LINESTRING (8.53681 47.38516, 8.53679 47.38521..."


In [78]:
gdf_abbiegeverbot.to_csv('./Daten/abbiegeverbote_ZH.csv', index=False)